## Лекция 2  BM5    

### TfidfVectorizer

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        |  слово1  |  слово2  |  слово3  |  слово4
# текст1 |   0.6    |    0.5   |   0.6    |    0
# текст2 |   0      |    0.6   |   0.8    |    0
# текст3 |   0.9    |    0.4   |   0      |    0
# текст4 |   0      |    0     |   0      |    1
 
# чтобы получить сгенерированный словарь, из приведенной структуры TfidfVectorizer
# порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']
 
# чтобы узнать индекс токена в словаре
vectorizer.vocabulary_.get('слово3') # вернет 2
 
# показать матрицу
X.toarray()
 
# теперь можно быстро подсчитать вектор для нового документа
new_doc = vectorizer.transform(['слово1 слово4 слово4']).toarray()  # результат [[0.36673901, 0, 0, 0.93032387]]
 

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [32]:
### реализуйте эту функцию ранжирования 
from math import log

k = 2.0
b = 0.75

def bm25(corpus, document, query):
    l_d = len(document.split())
    avgdl = get_av_len_doc(corpus)
    bm25_score = 0
    for word in query.split():
        idf = get_idf(word, corpus)
        tf = get_tf(word, document)
        bm25_score += idf * ((tf * (k + 1) ) / (tf + k * (1 - b + b * l_d / avgdl)))
    return bm25_score

In [33]:
def get_av_len_doc(corpus):
    n_words = 0
    n_docs = 0
    for doc in corpus:
        n_words += len(doc.split())
        n_docs += 1
    avgdl = n_words/n_docs
    return avgdl

In [34]:
def get_idf(word, corpus):
    N = len(corpus)
    nqi = 0
    for doc in corpus:
        if word in doc:
            nqi += 1
    if nqi == 0:
        nqi = 0.00000001
    idf = log(N - nqi + 0.5 / nqi + 0.5 )
    return idf

In [35]:
def get_tf(word, document):
    tf = 0
    for token in document.split():
        if token == word:
            tf += 1
    return tf

In [39]:
corpus = [
    'турция',
    'нужна справка срочно'
]
document1 = 'нужна справка срочно'
document2 = 'турция'
query = 'быстрая справка'

In [40]:
bm25(corpus, document1, query)

0.5545177444479562

In [41]:
bm25(corpus, document2, query)

0.0

In [ ]:
Действительно, метрика находит ближайший документ

In [ ]:
В такой реализации метрика максимально наглядна, но и максимально неоптимальна,
поэтому в заданиях я буду использовать реализацию из библиотеки rank_bm25

### __Задача 1__:    
Реализуйте поиск с метрикой *TF-IDF* через умножение матрицы на вектор.
Что должно быть в реализации:
- проиндексированная база, где каждый документ представлен в виде вектора TF-IDF
- функция перевода входяшего запроса в вектор по метрике TF-IDF
- ранжирование докуменов по близости к запросу по убыванию

В качестве корпуса возьмите корпус вопросов в РПН по Covid2019. Он состоит из:
> файл **answers_base.xlsx** - база ответов, у каждого ответа есть его номер, тематика и примеры вопросов, которые могут быть заданы к этому ответу. Сейчас проиндексировать надо именно примеры вопросов в качестве документов базы. Понимаете почему?

> файл **queries_base.xlsx** - вопросы юзеров, к каждому из которых проставлен номер верного ответа из базы. Разделите эти вопросы в пропорции 70/30 на обучающую (проиндексированную как база) и тестовую (как запросы) выборки. 


In [ ]:
Загружу всё необходимое: стопслова и предобработанные запросы

In [44]:
import re
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

def preprocess(text):
    text = text.lower()
    text = re.sub('[^а-яё]', ' ', text)
    text = [token for token in text.split() if token not in russian_stopwords]
    text = ' '.join(text)
    return text

In [45]:
queries_base = pd.read_csv('X_train_NER_preproc.csv')
tokens_corpus = queries_base.clean_text.tolist()

In [ ]:
Индексация базы

In [46]:
def query_base_indexing_tfidf():
    global tfidf_matrix, vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_corpus = tokens_corpus
    tfidf = vectorizer.fit_transform(tfidf_corpus)
    tfidf_matrix = tfidf.toarray()

In [47]:
%%time
query_base_indexing_tfidf()

CPU times: user 119 ms, sys: 29.9 ms, total: 149 ms
Wall time: 167 ms


In [ ]:
Поиск по базе

In [49]:
def tfidf_search(query):
    query = preprocess(query)
    query_tdidf_vector = vectorizer.transform([query]).toarray()
    cosine_similarities = linear_kernel(query_tdidf_vector, tfidf_matrix).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-5:-1]
    answer_doc = related_docs_indices[0]
    answer_id = queries_base['answer_id'][answer_doc]
    return answer_id

In [50]:
%%time
tfidf_search('я вернулся из стамбула, мне нужно сдать анализ на коронавирус?')

CPU times: user 40.4 ms, sys: 15.6 ms, total: 55.9 ms
Wall time: 46.7 ms


308

In [ ]:
Ответ верный, поиск быстрый

### __Задача 2__:    
Аналогичная задаче1 с другой метрикой 

Реализуйте поиск с метрикой *BM25* через умножение матрицы на вектор. Что должно быть в реализации:

- проиндексированная база, где каждый документ представлен в виде вектора BM25
- функция перевода входяшего запроса в вектор по метрике BM25
- ранжирование докуменов по близости к запросу по убыванию

In [ ]:
Индексация базы

In [51]:
def query_base_indexing_bm25():
    global bm25
    tokenized_corpus = [doc.split() for doc in tokens_corpus]
    bm25 = BM25Okapi(tokenized_corpus)

In [52]:
%%time
query_base_indexing_bm25()

CPU times: user 53.5 ms, sys: 6.17 ms, total: 59.7 ms
Wall time: 60.1 ms


In [ ]:
Поиск по базе

In [53]:
def bm25_search(query):
    query = preprocess(query)
    tokenized_query = query.split()
    answer_text = bm25.get_top_n(tokenized_query, tokens_corpus, n=1)
    answer_id = queries_base[queries_base['clean_text'] == answer_text[0]].iloc[0]['answer_id']
    return answer_id

In [54]:
%%time
bm25_search('я вернулся из стамбула, мне нужно сдать анализ на коронавирус?')

CPU times: user 9.2 ms, sys: 3.3 ms, total: 12.5 ms
Wall time: 16.8 ms


308

In [ ]:
Ответ верный, поиск ещё быстрее!